In [4]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import csv
import warnings

In [5]:
warnings.filterwarnings("ignore")

In [6]:
def plot_columns(df):
    colors = plt.cm.tab20(range(df.shape[1]))  

    for i, column in enumerate(df.columns):
        color = colors[i % 24]

        plt.figure()
        plt.plot(df.index, df[column], color=color)
        plt.xlabel('Ano')
        plt.ylabel('Valor')
        plt.title(column)

    plt.show()

In [7]:
def centered_moving_average(column, window_action, window_size):
    start_index, end_index = window_action
    data = column.to_numpy()

    data_length = len(data)

    for i in range(start_index, end_index + 1):
        if not np.isnan(data[i]):
            init = data[i]
            start = max(0, i - window_size)
            end = min(data_length, i + window_size + 1)
            window = data[start:end]
            valid_values = [value for value in window if not pd.isna(value)]
            if len(valid_values) > 0:
                data[i] = np.mean(valid_values)

    modified_column = pd.Series(data, name=column.name)

    return modified_column

In [8]:
# Polynomial Interpolation data inputation method
def four_ends_filler(column):
    if  pd.api.types.is_numeric_dtype(column):
        for i in range(4):
            column = first_filler(column, i, len(column) - 1 - i)
            column = last_filler(column, i, len(column) - 1 - i)
    return column

def four_ends_linear_filler(column):
    if  pd.api.types.is_numeric_dtype(column):
        size = len(column) - 1
        column = first_filler(column, 0, size)
        column = last_filler(column, 0, size)
        for i in range(0, 4):
            value = column.at[i]
            if i > 0 and (pd.isnull(value) or np.isnan(value) or (value == '')):
                start_first = prev_valid_index(column, i)
                end_first = next_valid_index(column, i)
                gap = list(range(start_first + 1, end_first))
                linear_a = (column.at[end_first] - column.at[start_first])/(end_first - start_first)
                linear_b = column.at[start_first] - linear_a*start_first
                column.at[i] = linear_a*i + linear_b
            value = column.at[size - 4 + i]
            if i > 0 and (pd.isnull(value) or np.isnan(value) or (value == '')):
                start_last = prev_valid_index(column, size - 4 + i)
                end_last = next_valid_index(column, size - 4 + i)
                gap = list(range(start_last + 1, end_last))
                linear_a = (column.at[end_last] - column.at[start_last])/(end_last - start_last)
                linear_b = column.at[start_last] - linear_a*start_last
                column.at[size - 4 + i] = linear_a*i + linear_b
    return column

def first_filler(column, init, end):
    if pd.isnull(column.at[init]) or np.isnan(column.at[init]):
        first_valid_index = 0
        while first_valid_index <= end and (pd.isnull(column.at[first_valid_index]) or 
                                             np.isnan(column.at[first_valid_index])):
            first_valid_index += 1
        if first_valid_index > end:
            column.at[init] = 0.0
        else:
            column.at[init] = column.at[first_valid_index]
    return column
    
def last_filler(column, init, end):
    if pd.isnull(column.at[end]) or np.isnan(column.at[end]):
        last_valid_index = len(column) -1
        while last_valid_index >= init and (pd.isnull(column.at[last_valid_index]) or 
                                         np.isnan(column.at[last_valid_index])):
            last_valid_index -= 1
        if last_valid_index < init:
            column.at[end] = 0.0
        else:
            column.at[end] = column.at[last_valid_index]
    return column
  
def neareast_four_indexes(column, start, end):
    return prev_four_valid_indexes(column, start) + next_four_valid_indexes(column, end)

def prev_four_valid_indexes(column, pos):
    prev_index = pos
    index_prev_list = [0]*4
    size = len(column) - 1
    if pos > 0 and pos < size:
        for i in range(4):
            index_prev_list[3 - i] = prev_index
            prev_index -= 1
            while prev_index >= 0 and (pd.isnull(column.at[prev_index]) or np.isnan(column.at[prev_index])):
                prev_index -= 1
    return index_prev_list  

def next_four_valid_indexes(column, pos):
    next_index = pos
    index_next_list = [0]*4
    size = len(column) - 1
    if pos > 0 and pos < size:
        for i in range(4):
            index_next_list[i] = next_index
            next_index += 1
            while next_index <= size and (pd.isnull(column.at[next_index]) or np.isnan(column.at[next_index])):
                next_index += 1
    return index_next_list  

def prev_valid_index(column, pos):
    prev_index = pos
    size = len(column) -1
    if pos >= 0 and pos < size:
        prev_index -= 1
        while prev_index >= 0 and (pd.isnull(column.at[prev_index]) or np.isnan(column.at[prev_index])):
            prev_index -= 1
    return prev_index  

def next_valid_index(column, pos):
    next_index = pos
    size = len(column) -1
    if pos >= 0 and pos < size:
        next_index += 1
        while next_index <= size and (pd.isnull(column.at[next_index]) or np.isnan(column.at[next_index])):
            next_index += 1
    return next_index

def interpolating_polynomial(nearest_indexes, nearest_values):
    coeffs = np.polyfit(nearest_indexes, nearest_values, 3)
    return np.poly1d(coeffs)

def polynomial_filler(column):
    size = len(column) -1
    if column.isnull().any() and column.isna().any():
        for i, value in enumerate(column):
            if (i > 3 and i < size - 3) and (pd.isnull(value) or np.isnan(value) or (value == '')):
                nearest_values = []
                start = prev_valid_index(column, i)
                end = next_valid_index(column, i)
                gap = list(range(start + 1, end))
                nearest_indexes = neareast_four_indexes(column, start, end)
                for index in nearest_indexes:
                    nearest_values.append(column.at[index])
                polynomial = interpolating_polynomial(nearest_indexes, nearest_values)
                for x in gap:
                    column.at[x] = polynomial(x)
                plot_polynomial(polynomial, nearest_indexes[0], nearest_indexes[len(nearest_indexes) - 1], 50, gap, nearest_indexes, column.name)
    return column

def plot_polynomial(polynomial, start, end, precision, interpolated_list, nearest_indexes, column_name):
    x = np.linspace(start, end, precision)
    y = polynomial(x)

    plt.figure()

    plt.plot(x, y)

    interpolated_x = interpolated_list  
    interpolated_y = polynomial(interpolated_x)
    plt.scatter(interpolated_x, interpolated_y, color='red')
    interpolated_x = nearest_indexes  
    interpolated_y = polynomial(interpolated_x)
    plt.scatter(interpolated_x, interpolated_y, color='green')
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(column_name)

    plt.show()
    
def replace_negative_with_float_zeros(x):
    return 0.0 if x < 0 else x

In [9]:
def read_csv_replace_missing_by_polynomial(file_path, skiped_rows, skiped_columns):
    with open(file_path, 'r') as f:
        
        df = pd.read_csv(file_path, skiprows=skiped_rows)
        
        i = 0
        for col in df.columns: # Elimina missing values por médias e transforma em float
            if i > skiped_columns:
                df[col] = df[col].replace('-9999', np.nan)
                df[col] = df[col].replace('-', np.nan)
                df[col] = df[col].replace('', np.nan).astype(float)
                df[col] = four_ends_linear_filler(df[col])
                df[col] = centered_moving_average(df[col], (0, len(df[col]) - 1), 24) #2 anos
                df[col] = polynomial_filler(df[col])
            i += 1
        return df

In [10]:
df = read_csv_replace_missing_by_polynomial('pib_sao_paulo.csv', 0, 0)
df.head(len(df))

,Ano,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado
0,1985,3.203715e+08,4.570399e+07,23.485383,8.298747e+08
1,1986,3.615961e+08,4.682616e+07,23.508608,8.445674e+08
2,1987,4.054788e+08,4.758042e+07,23.528358,8.577608e+08
3,1988,4.514324e+08,4.836571e+07,23.538600,8.699503e+08
4,1989,5.003781e+08,4.903840e+07,23.557961,8.828795e+08
5,1990,5.521405e+08,4.938077e+07,23.602759,8.954005e+08
6,1991,6.045921e+08,4.994044e+07,23.672765,9.078550e+08
7,1992,6.585073e+08,5.029761e+07,23.735575,9.193863e+08
8,1993,7.126441e+08,5.042863e+07,23.818311,9.310825e+08
9,1994,7.669935e+08,5.033744e+07,23.866202,9.409031e+08


In [11]:
df_year = df.copy()
df_year.set_index('Ano', inplace=True)
df_year

,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado
Ano,,,,
1985,3.203715e+08,4.570399e+07,23.485383,8.298747e+08
1986,3.615961e+08,4.682616e+07,23.508608,8.445674e+08
1987,4.054788e+08,4.758042e+07,23.528358,8.577608e+08
1988,4.514324e+08,4.836571e+07,23.538600,8.699503e+08
1989,5.003781e+08,4.903840e+07,23.557961,8.828795e+08
1990,5.521405e+08,4.938077e+07,23.602759,8.954005e+08
1991,6.045921e+08,4.994044e+07,23.672765,9.078550e+08
1992,6.585073e+08,5.029761e+07,23.735575,9.193863e+08
1993,7.126441e+08,5.042863e+07,23.818311,9.310825e+08


In [12]:
df_2014 = df_year[df_year.index >= 2014]
df_2014

,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado
Ano,,,,
2014,1.197826e+09,4.735435e+07,24.001434,1.003012e+09
2015,1.206909e+09,4.689336e+07,23.951008,1.001922e+09
2016,1.212819e+09,4.650979e+07,23.922375,1.001305e+09
2017,1.214213e+09,4.632537e+07,23.910733,1.000929e+09
2018,1.211766e+09,4.629030e+07,23.902108,1.000151e+09
2019,1.204757e+09,4.635890e+07,23.895877,9.992364e+08
2020,1.190468e+09,4.655413e+07,23.874352,9.969671e+08
2021,1.171137e+09,4.681715e+07,23.903240,9.959823e+08
2022,1.147642e+09,4.713421e+07,23.936645,9.946419e+08


In [10]:
df_2014.to_csv('2014_01_pib.csv')

In [10]:
df_2008 = df_year[df_year.index >= 2008]
df_2008

,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado
Ano,,,,
2008,1.092396e+09,4.853082e+07,24.143787,9.946828e+08
2009,1.098669e+09,4.873392e+07,24.141551,9.947096e+08
2010,1.124010e+09,4.884708e+07,24.146600,9.985217e+08
2011,1.146637e+09,4.854680e+07,24.134292,1.000762e+09
2012,1.166721e+09,4.823983e+07,24.089660,1.002188e+09
2013,1.184231e+09,4.786423e+07,24.041932,1.003187e+09
2014,1.197826e+09,4.735435e+07,24.001434,1.003012e+09
2015,1.206909e+09,4.689336e+07,23.951008,1.001922e+09
2016,1.212819e+09,4.650979e+07,23.922375,1.001305e+09


In [11]:
df_2008.to_csv('2008_01_pib.csv')